In [54]:
"""
    
    parameters
    ----------
    dataset: dict
        dictionary of datasets to use. it will be
        mens_data or womens_data.
        
    num_year: int
        Number of years of tournament history you want to look at; default is 5 which include
        2019, 2020, 2021, 2022, and 2023 for the 2024 season
    curr_year: int
        The year we'll be trying to predict
        
    returns
    -------
    tournament_record_stats: pd.DataFrame
        dataframe with count of wins, losses for each team for tournament games only;
        value will be -1 for teams that don't have a win or loss in the postseason
    """
def get_recent_tournament_record(dataset, num_years=5, curr_year=2024):
    gender = list(dataset.keys())[0][0]
    df = dataset[f"{gender}NCAATourneyCompactResults"]
    
    """
    for each year in correct range, count the number of times a team id appears in
    wTeamID and LTeamID
    """
    starting_year = curr_year-num_years
    yearly_df=df.query('Season>=@starting_year')
    win_count_df = yearly_df.groupby("WTeamID")['WLoc'].count().reset_index()
    loss_count_df = yearly_df.groupby("LTeamID")['WLoc'].count().reset_index()
    
    
    win_df = win_count_df.rename(columns={"WTeamID": "TeamID"})
    win_df = win_df.rename(columns={"WLoc": "numTournWins"})
    lose_df = loss_count_df.rename(columns={"LTeamID": "TeamID"})
    lose_df = lose_df.rename(columns={"WLoc": "numTournLoss"})
    
    merged = pd.merge(win_df, lose_df, on='TeamID', how='outer' )
    # merge with alll teams!
    merged2 = pd.merge(merged, data['MTeams']['TeamID'],on='TeamID', how='outer')
    # replace NaN with -1
    final = merged2.fillna(-1)
    
    return final
    
get_recent_tournament_record(mens_data)

,TeamID,numTournWins,numTournLoss
0,1101,1.0,2.0
1,1102,-1.0,-1.0
2,1103,-1.0,1.0
3,1104,4.0,3.0
4,1105,-1.0,-1.0
...,...,...,...
373,1474,-1.0,-1.0
374,1475,-1.0,-1.0
375,1476,-1.0,-1.0
376,1477,-1.0,-1.0


This is where I will write feature functions.

Other ideas from me:
- Number of players over 7ft tall
- Conference they're in and some sort of conference ranking
- Is there a way to get injury report - something simple like how many people injured or change in injured players from beginning of season